In [0]:

USE CATALOG dibyendu_catalog;
CREATE STREAMING LIVE TABLE sales
AS SELECT *, current_timestamp() AS ingestion_date FROM stream(dibyendu_catalog.delta_demo_pipeline_schema.sales);

In [0]:

USE CATALOG dibyendu_catalog;
CREATE STREAMING LIVE TABLE silver_sales
(
  CONSTRAINT valid_order_id EXPECT(order_id is NOT NULL) ON VIOLATION DROP ROW
)
AS SELECT distinct * FROM stream(LIVE.sales);

In [0]:
USE CATALOG dibyendu_catalog;
CREATE STREAMING LIVE TABLE customes
AS SELECT *, current_timestamp() AS ingestion_date FROM stream(dibyendu_catalog.delta_demo_pipeline_schema.customers);

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE customer_silver;

APPLY CHANGES INTO
  Live.customer_silver
FROM
  stream(Live.customes)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum, ingestion_date)
STORED AS
  SCD TYPE 2;

In [0]:
create streaming table customer_silver_active as
select customer_id, customer_name,customer_email,customer_city,customer_state from stream(live.customer_silver)
where __END_AT is NULL

In [0]:
USE CATALOG dibyendu_catalog;
CREATE STREAMING LIVE TABLE products
AS SELECT *, current_timestamp() AS ingestion_date FROM stream(dibyendu_catalog.delta_demo_pipeline_schema.products);

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE products_silver;

APPLY CHANGES INTO
  Live.products_silver
FROM
  stream(Live.products)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
APPLY AS TRUNCATE WHEN
  operation = "TRUNCATE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum, ingestion_date)
STORED AS
  SCD TYPE 1;

In [0]:
create live table total_sales_customer_gold_MV as 
SELECT 
    c.customer_id,
    c.customer_name,
    round(SUM(s.total_amount)) AS total_sales,
    SUM(s.discount_amount) AS total_discount
FROM LIVE.silver_sales s
JOIN LIVE.customer_silver_active c
    ON s.customer_id = c.customer_id
GROUP BY c.customer_id, c.customer_name
ORDER BY total_sales DESC;

In [0]:
create live table total_sales_category_gold_MV as
SELECT 
    p.product_category,
    round(SUM(s.total_amount)) AS total_sales
FROM LIVE.silver_sales s
JOIN live.products_silver p
    ON s.product_id = p.product_id
GROUP BY p.product_category
ORDER BY total_sales DESC;